<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка" data-toc-modified-id="Подготовка-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка</a></span></li><li><span><a href="#Обучение" data-toc-modified-id="Обучение-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обучение</a></span></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Выводы</a></span></li></ul></div>

# Проект для «Викишоп»

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.


In [2]:
RANDOM_STATE = 42

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [4]:
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn import linear_model
from sklearn.pipeline import Pipeline
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score, make_scorer

In [5]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [6]:
import lightgbm as lgbm

In [9]:
import nltk
from nltk.corpus import stopwords as nltk_stopwords

In [113]:
import random
import re
from multiprocessing import Pool

In [10]:
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [12]:
from tqdm import tqdm

In [13]:
tqdm.pandas()

## Подготовка

- Лемматизация
- Векторизация (TfidfVectorizer) с учетом стоп-слов

In [121]:
#Загрузим данные
#p = 0.7 
try:
    df = pd.read_csv('/datasets/toxic_comments.csv') #skiprows=lambda i: i>0 and random.random() > p)

In [122]:
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159292 entries, 0 to 159291
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   Unnamed: 0  159292 non-null  int64 
 1   text        159292 non-null  object
 2   toxic       159292 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 3.6+ MB


,Unnamed: 0,text,toxic
0,0,Explanation\nWhy the edits made under my usern...,0
1,1,D'aww! He matches this background colour I'm s...,0
2,2,"Hey man, I'm really not trying to edit war. It...",0
3,3,"""\nMore\nI can't make any real suggestions on ...",0
4,4,"You, sir, are my hero. Any chance you remember...",0


In [124]:
df['toxic'].sum()/df.shape[0]

0.10161213369158527

In [125]:
lemmatizer = WordNetLemmatizer()

In [126]:
def wordnet_pos(word):
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {'J':wordnet.ADJ,
               'N':wordnet.NOUN,
               'V':wordnet.VERB,
               'R':wordnet.ADV}
    return tag_dict.get(tag, wordnet.NOUN)

In [127]:
def lemmatize(text):
    word_list = word_tokenize(text)
    text = [lemmatizer.lemmatize(w, wordnet_pos(w)) for w in word_list]
    return ' '.join(text)

In [128]:
def clear_text(text):
    clear_list = re.sub(r"[^-/().&' \w]|_",' ', text)
    clear_text = "".join(clear_list)
    clear_list = clear_text.split()
    return " ".join(clear_list)

In [129]:
print(lemmatize('My puppies barked all night with birds quietly singing @ !\ ^ 345'))
print(clear_text('My puppies barked all night with birds quietly singing @ !\ ^ 345'))

My puppy bark all night with bird quietly sing @ ! \ ^ 345
My puppies barked all night with birds quietly singing 345


In [130]:
%%time
df['text'] = df['text'].progress_apply(lemmatize)

100%|████████████████████████████████████████████████████████████████████████| 159292/159292 [3:38:27<00:00, 12.15it/s]


Wall time: 3h 38min 28s


In [134]:
%%time
df['text'] = df['text'].progress_apply(clear_text)
df['text'] = df['text'].str.lower()

100%|███████████████████████████████████████████████████████████████████████| 159292/159292 [00:04<00:00, 32750.95it/s]


Wall time: 5.04 s


In [135]:
df['text'].head()

0    explanation why the edits make under my userna...
1    d'aww he match this background colour i 'm see...
2    hey man i 'm really not try to edit war . it '...
3    more i ca n't make any real suggestion on impr...
4    you sir be my hero . any chance you remember w...
Name: text, dtype: object

In [136]:
#сохраним лемматизированные твиты на всякий случай df.to_csv('lemmatized_tweets.csv')

In [137]:
#Разделим выборки
target = df['toxic']
features = df['text']
features_train, features_test, target_train, target_test = \
train_test_split(features,target, test_size=0.25, shuffle = True, random_state=RANDOM_STATE)

In [138]:
%%time
stopwords = set(nltk_stopwords.words('english'))
count_tf_idf = TfidfVectorizer(stop_words=stopwords) 
features_train = count_tf_idf.fit_transform(features_train)

Wall time: 6.93 s


In [139]:
features_test = count_tf_idf.transform(features_test)

In [140]:
print(features_train.shape)
print(features_test.shape)

(119469, 148116)
(39823, 148116)


## Обучение

In [141]:
#словари параметров для гридсерчей

param_grid_dt = {'max_depth': [None] + [i for i in range(2, 8)], 'class_weight' : ['balanced']}

param_grid_gb = {"max_depth": [None] + [i for i in range(2, 8)],
    "num_leaves": range(10, 100, 10), 'class_weight' : ['balanced']}

#param_grid_rf = {'max_depth': [None] + [i for i in range(2, 10)], \
                 #'n_estimators':(10, 100, 10), 'class_weight' : ['balanced']}

param_grid_lr = {'C': [0.0001,0.0005,0.001,0.1,1,10],
    'solver' : ['newton-cg', 'lbfgs', 'liblinear'],
    'penalty':['l2'],
    'class_weight' : ['balanced'],
                'max_iter' : [1000]}

In [60]:
#f1 = make_scorer(f1_score , average='macro')

In [142]:
%%time
lr_search = GridSearchCV(linear_model.LogisticRegression(), cv = 5, param_grid = param_grid_lr, \
                         scoring='f1') #verbose=10)
lr_search.fit(features_train, target_train)

print('F1 score логистической регресии на обучающей выборке на кросс-валидации:',\
      lr_search.best_score_)

F1 score логистической регресии на обучающей выборке на кросс-валидации: 0.759492160350788
Wall time: 3min 37s


In [ ]:
%%time
dt_search = GridSearchCV(DecisionTreeClassifier(random_state = RANDOM_STATE), cv = 5, \
                         param_grid = param_grid_dt,\
                         scoring='f1') #verbose=10)

dt_search.fit(features_train, target_train)
print('F1 score дерева решений на обучающей выборке на кросс-валидации:', dt_search.best_score_)

In [ ]:
%%time
gb_search = GridSearchCV(lgbm.LGBMClassifier(random_seed=RANDOM_STATE), cv = 5, \
                         param_grid = param_grid_dt, scoring='f1') # verbose=10)
gb_search.fit(features_train, target_train)

print('F1 score градиентного бустинга на обучающей выборке на кросс-валидации:', gb_search.best_score_)

**Вывод по отбору модели:** лучше всех себя показала себя логистическая регрессия со значением F1 = 0.759. Протестируем эту модель на тестовой выборке.

## Выводы

In [145]:
f1_test = f1_score(lr_search.best_estimator_.predict(features_test),\
                                  target_test)
print('F1 score модели на тестовой выборке:', f1_test)

F1 score модели на тестовой выборке: 0.7610392636607247


Мы получили обученную логистическую регрессию с F1 score на тестовой выборке 0.7610392636607247 - можно отсеивать твиты.